In [2]:
import os
import sys
from pathlib import Path

CURRENT_DIRECTORY = Path(os.getcwd())
ROOT_DIRECTORY = (CURRENT_DIRECTORY / "..").absolute().resolve()

print(f"Current directory: {CURRENT_DIRECTORY}")
print(f"Root directory: {ROOT_DIRECTORY}")

sys.path.append(str(ROOT_DIRECTORY))

Current directory: /home/ubuntu/arga-arc/tf_coder
Root directory: /home/ubuntu/arga-arc


In [3]:
import typing as t
import json
from pprint import pprint
from dataclasses import dataclass
import tensorflow as tf
import numpy as np
import math
from config import CONFIG
from openai import OpenAI
import re
from collections import Counter
import random

pprint(CONFIG.__dict__.keys())

OPENAI = OpenAI(api_key=CONFIG.OPENAI_SECRET_KEY, organization=CONFIG.OPENAI_ORGANIZATION)

2024-03-20 17:08:11.691103: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-20 17:08:11.709713: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 17:08:11.815765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 17:08:11.815827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 17:08:11.832511: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

dict_keys(['OPENAI_SECRET_KEY', 'OPENAI_ORGANIZATION'])


## parsing the dataset

In [39]:
class OutputJSON(t.TypedDict):
    task_id: str
    completions: t.List[str]
    coverage_percentage: float
    description: str
    tf_operators: t.Dict[str, int]
    total_covered: int
    total_in_target: int

class ExamplesJSON(t.TypedDict):
    inputs: str
    outputs: str

class TaskJSON(t.TypedDict):
    constants: str
    description: str
    name: str
    source: str
    target_program: str
    examples: ExamplesJSON

DATASET_FILE = CURRENT_DIRECTORY / "tfcoder_dataset.json"
DATASET: t.List[TaskJSON] = json.loads(DATASET_FILE.read_text())

print(f"Loaded {len(DATASET)} tasks from {DATASET_FILE}")

Loaded 72 tasks from /home/ubuntu/arga-arc/tf_coder/tfcoder_dataset.json


In [40]:
IN_CONTEXT_TASK_JSONS = random.sample(DATASET, 3)
pprint(IN_CONTEXT_TASK_JSONS)

[{'constants': '[]',
  'description': 'Divide each row by the sum of that row',
  'examples': {'inputs': '[[[0.0, 1.0, 0.0, 0.0], [0.0, 1.0, 1.0, 0.0], [1.0, '
                         '1.0, 1.0, 1.0]],]',
               'outputs': '[[0.0, 1.0, 0.0, 0.0],    [0.0, 0.5, 0.5, 0.0],    '
                          '[0.25, 0.25, 0.25, 0.25]]'},
  'name': 'google_02',
  'source': 'Real task encountered by Googler, 11/01/2018',
  'target_program': 'tf.divide(in1, tf.expand_dims(tf.reduce_sum(in1, axis=1), '
                    '1))'},
 {'constants': '[]',
  'description': 'gather the marked elements',
  'examples': {'inputs': '[[10, 20, 0, 40, 0, 30],[1, 1, 0, 1, 0, 1],]',
               'outputs': '[10, 20, 40, 30]'},
  'name': 'google_10',
  'source': None,
  'target_program': 'tf.boolean_mask(in1, tf.cast(in2, tf.bool))'},
 {'constants': '[]',
  'description': 'swap the first two dimensions of the tensor',
  'examples': {'inputs': '[[[[8, 4, 6], [2, 12, 3]], [[11, 12, 5], [9, 12, '
       

In [41]:
TASK_JSONS = {
    task["name"]: task for task in DATASET if task not in IN_CONTEXT_TASK_JSONS
}
pprint(len(TASK_JSONS.values()))

69


In [42]:
@dataclass
class Example:
    inputs: t.List[np.ndarray]
    output: t.Union[np.ndarray, tf.SparseTensor]

    @classmethod
    def from_json(cls, examples: ExamplesJSON):
        try:
            evaluated_inputs = eval(examples["inputs"])
            if isinstance(evaluated_inputs, list):
                inputs = [np.array(i) for i in evaluated_inputs]
            else:
                inputs = [evaluated_inputs]
        except Exception as e:
            print(f"Error evaluating inputs: {e}")
            print(f"Inputs: {examples['inputs']}")
            raise e

        try:
            evaluated_outputs = eval(examples["outputs"])
            if isinstance(evaluated_outputs, list):
                outputs = np.array(evaluated_outputs)
            elif isinstance(evaluated_outputs, tf.SparseTensor):
                outputs = evaluated_outputs
            elif isinstance(evaluated_outputs, tf.Tensor):
                outputs = evaluated_outputs.numpy()
            else:
                outputs = evaluated_outputs
        except Exception as e:
            print(f"Error evaluating outputs: {e}")
            print(f"Outputs: {examples['outputs']}")
            raise e

        return cls(inputs, outputs)
    
    def toJSON(self):
        return {
            "inputs": [i.tolist() for i in self.inputs],
            "output": self.output.tolist()
        }

for name, task in TASK_JSONS.items():
    task["parsed_examples"] = Example.from_json(task["examples"])

pprint(TASK_JSONS["google_03"])


{'constants': '[]',
 'description': 'Slice the first dimension of a SparseTensor',
 'examples': {'inputs': 'tf.SparseTensor(indices=[[0, 0, 0], [0, 1, 1], [1, 1, '
                        '1], [1, 1, 2]], values=[1., 1., 1., 1.], '
                        'dense_shape=[2, 2, 800])',
              'outputs': 'tf.SparseTensor(indices=[[0, 0, 0], [0, 1, 1]], '
                         'values=[1., 1.], dense_shape=[1, 2, 800])'},
 'name': 'google_03',
 'parsed_examples': Example(inputs=[SparseTensor(indices=tf.Tensor(
[[0 0 0]
 [0 1 1]
 [1 1 1]
 [1 1 2]], shape=(4, 3), dtype=int64), values=tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32), dense_shape=tf.Tensor([  2   2 800], shape=(3,), dtype=int64))],
                            output=SparseTensor(indices=tf.Tensor(
[[0 0 0]
 [0 1 1]], shape=(2, 3), dtype=int64), values=tf.Tensor([1. 1.], shape=(2,), dtype=float32), dense_shape=tf.Tensor([  1   2 800], shape=(3,), dtype=int64))),
 'source': 'Real task encountered by Googler, 11/01/20

## prompt

In [6]:
TFOPERATORS_STR = "tf.abs(x)\ntf.add(x, y)\ntf.add_n(inputs)\ntf.argmax(input, axis)\ntf.argmin(input, axis)\n"+\
"tf.argsort(values, axis, stable=True)\ntf.argsort(values, axis, direction='DESCENDING', stable=True)\ntf.boolean_mask(tensor, mask)\ntf.broadcast_to(input, shape)\n"+\
"tf.cast(x, dtype)\ntf.clip_by_value(t, clip_value_min, clip_value_max)\ntf.concat(values, axis)\ntf.constant(value)\ntf.constant(value, dtype)\ntf.divide(x, y)\n"+\
"tf.equal(x, y)\ntf.exp(x)\ntf.expand_dims(input, axis)\ntf.eye(num_rows)\ntf.eye(num_rows, num_columns)\ntf.eye(num_rows, dtype)\ntf.fill(dims, value)"+\
"tf.gather(params, indices)\ntf.gather(params, indices, axis, batch_dims)\ntf.gather_nd(params, indices)\ntf.gather_nd(params, indices, batch_dims)\ntf.greater(x, y)\n"+\
"tf.greater_equal(x, y)\ntf.math.bincount(arr)\ntf.math.ceil(x)\ntf.math.count_nonzero(input)\ntf.math.count_nonzero(input, axis)\ntf.math.cumsum(x, axis)\n"+\
"tf.math.cumsum(x, axis, exclusive=True)\ntf.math.divide_no_nan(x, y)\ntf.math.floor(x)\ntf.math.log(x)\ntf.math.logical_and(x, y)\ntf.math.logical_not(x)"+\
"tf.math.logical_or(x, y)\ntf.math.logical_xor(x, y)\ntf.math.negative(x)\ntf.math.reciprocal(x)\ntf.math.reciprocal_no_nan(x)\ntf.math.segment_max(data, segment_ids)\n"+\
"tf.math.segment_mean(data, segment_ids)\ntf.math.segment_min(data, segment_ids)\ntf.math.segment_prod(data, segment_ids)\ntf.math.segment_sum(data, segment_ids)\n"+\
"tf.math.squared_difference(x, y)\ntf.math.top_k(input, k)\ntf.math.unsorted_segment_max(data, segment_ids, num_segments)\ntf.math.unsorted_segment_mean(data, segment_ids, num_segments)\n"+\
"tf.math.unsorted_segment_min(data, segment_ids, num_segments)\ntf.math.unsorted_segment_prod(data, segment_ids, num_segments)\ntf.math.unsorted_segment_sum(data, segment_ids, num_segments)\n"+\
"tf.matmul(a, b)\ntf.maximum(x, y)\ntf.minimum(x, y)\ntf.multiply(x, y)\ntf.not_equal(x, y)\ntf.one_hot(indices, depth)\ntf.ones(shape)\ntf.ones_like(input)\n"+\
"tf.pad(tensor, paddings, mode='CONSTANT')\ntf.pad(tensor, paddings, mode='CONSTANT', constant_values)\ntf.pad(tensor, paddings, mode='REFLECT')\n"+\
"tf.pad(tensor, paddings, mode='SYMMETRIC')\ntf.range(start)\ntf.range(start, limit, delta)\ntf.reduce_any(input_tensor, axis)\ntf.reduce_all(input_tensor, axis)\n"+\
"tf.reduce_max(input_tensor)\ntf.reduce_max(input_tensor, axis)\ntf.reduce_mean(input_tensor)\n"+\
"tf.reduce_mean(input_tensor, axis)\ntf.reduce_min(input_tensor)\ntf.reduce_min(input_tensor, axis)\n"+\
"tf.reduce_prod(input_tensor, axis)\ntf.reduce_sum(input_tensor)\ntf.reduce_sum(input_tensor, axis)\n"+\
"tf.repeat(input, repeats)\ntf.repeat(input, repeats, axis)\ntf.reshape(tensor, shape)\n"+\
"tf.reverse(tensor, axis)\ntf.roll(input, shift, axis)\ntf.round(x)\ntf.scatter_nd(indices, updates, shape)\n"+\
"tf.searchsorted(sorted_sequence, values, side='left')\ntf.searchsorted(sorted_sequence, values, side='right')\n"+\
"tf.sequence_mask(lengths)\ntf.sequence_mask(lengths, maxlen)\ntf.shape(input)\ntf.sign(x)\n"+\
"tf.sort(values, axis)\ntf.sort(values, axis, direction='DESCENDING')\ntf.sqrt(x)\n"+\
"tf.square(x)\ntf.squeeze(input)\ntf.squeeze(input, axis)\ntf.stack(values, axis)\ntf.subtract(x, y)\n"+\
"tf.tensor_scatter_nd_update(tensor, indices, updates)\ntf.tensordot(a, b, axes)\ntf.tile(input, multiples)\n"+\
"tf.transpose(a)\ntf.transpose(a, perm)\ntf.unique_with_counts(x)\ntf.unstack(value, axis)\n"+\
"tf.where(condition)\ntf.where(condition, x, y)\ntf.zeros(shape)\ntf.zeros_like(input)"
SPARSETF_OPERATORS_STR = "tf.SparseTensor(indices, values, dense_shape)\ntf.sparse.add(a, b)\n"+\
"tf.sparse.concat(axis, sp_inputs)\ntf.sparse.expand_dims(sp_input, axis)\ntf.sparse.from_dense(tensor)\ntf.sparse.maximum(sp_a, sp_b)\n"+\
"tf.sparse.minimum(sp_a, sp_b)\ntf.sparse.reduce_max(sp_input, axis, output_is_sparse)\ntf.sparse.reduce_sum(sp_input, axis, output_is_sparse)\n"+\
"tf.sparse.reset_shape(sp_input)\ntf.sparse.reshape(sp_input, shape)\ntf.sparse.retain(sp_input, to_retain)\ntf.sparse.slice(sp_input, start, size)\n"+\
"tf.sparse.split(sp_input, num_split, axis)\ntf.sparse.to_dense(sp_input)\ntf.sparse.to_dense(sp_input, default_value)\n"+\
"tf.sparse.to_indicator(sp_input, vocab_size)\ntf.sparse.transpose(sp_input)\ntf.sparse.transpose(sp_input, perm)"

In [7]:
TFOPERATORS = [op.strip() for op in TFOPERATORS_STR.split("\n") if op.strip() != ""]

SPARSETF_OPERATORS = [op.strip() for op in SPARSETF_OPERATORS_STR.split("\n") if op.strip() != ""]

print(f"TF Operators: {len(TFOPERATORS)}, {TFOPERATORS[:5]}")
print(f"SparseTF Operators: {len(SPARSETF_OPERATORS)}, {SPARSETF_OPERATORS[:5]}")

TF Operators: 111, ['tf.abs(x)', 'tf.add(x, y)', 'tf.add_n(inputs)', 'tf.argmax(input, axis)', 'tf.argmin(input, axis)']
SparseTF Operators: 19, ['tf.SparseTensor(indices, values, dense_shape)', 'tf.sparse.add(a, b)', 'tf.sparse.concat(axis, sp_inputs)', 'tf.sparse.expand_dims(sp_input, axis)', 'tf.sparse.from_dense(tensor)']


In [8]:
TFOPERATORS_WEIGHT_ORDER = ['tf.cast(x, dtype)', 'tf.expand_dims(input, axis)', 'tf.constant(value)', 'tf.squeeze(input, axis)', 'tf.constant(value, dtype)', 'tf.equal(x, y)', 'tf.gather(params, indices)', 'tf.greater(x, y)', 'tf.matmul(a, b)', 'tf.maximum(x, y)', 'tf.multiply(x, y)', 'tf.reduce_max(input_tensor)', 'tf.reduce_max(input_tensor, axis)', 'tf.reduce_sum(input_tensor)', 'tf.reduce_sum(input_tensor, axis)', 'tf.tensordot(a, b, axes)', 'tf.transpose(a)', 'tf.where(condition)', 'tf.where(condition, x, y)', 'tf.add(x, y)', 'tf.boolean_mask(tensor, mask)', 'tf.divide(x, y)', 'tf.gather_nd(params, indices)', 'tf.one_hot(indices, depth)', 'tf.range(start)', 'tf.reshape(tensor, shape)', 'tf.square(x)', 'tf.subtract(x, y)', 'tf.tile(input, multiples)', 'tf.argmax(input, axis)', 'tf.greater_equal(x, y)', 'tf.minimum(x, y)', 'tf.sequence_mask(lengths)', 'tf.zeros_like(input)', 'tf.concat(values, axis)', 'tf.gather_nd(params, indices, batch_dims)', 'tf.ones_like(input)', 'tf.shape(input)', 'tf.stack(values, axis)', 'tf.squeeze(input)', 'tf.abs(x)', 'tf.argsort(values, axis, stable=True)', 'tf.eye(num_rows)', 'tf.fill(dims, value)', 'tf.gather(params, indices, axis, batch_dims)', 'tf.math.bincount(arr)', 'tf.math.segment_max(data, segment_ids)', 'tf.math.segment_sum(data, segment_ids)', 'tf.math.unsorted_segment_max(data, segment_ids, num_segments)', 'tf.math.unsorted_segment_sum(data, segment_ids, num_segments)', "tf.pad(tensor, paddings, mode='CONSTANT')", 'tf.reduce_any(input_tensor, axis)', 'tf.reduce_mean(input_tensor)', 'tf.reduce_mean(input_tensor, axis)', 'tf.reduce_min(input_tensor)', 'tf.reduce_min(input_tensor, axis)', 'tf.unstack(value, axis)', 'tf.zeros(shape)', 'tf.add_n(inputs)', 'tf.broadcast_to(input, shape)', 'tf.clip_by_value(t, clip_value_min, clip_value_max)', 'tf.math.ceil(x)', 'tf.math.cumsum(x, axis)', 'tf.math.floor(x)', 'tf.math.logical_and(x, y)', 'tf.math.logical_or(x, y)', 'tf.not_equal(x, y)', 'tf.ones(shape)', 'tf.reduce_all(input_tensor, axis)', 'tf.sequence_mask(lengths, maxlen)', 'tf.tensor_scatter_nd_update(tensor, indices, updates)', 'tf.transpose(a, perm)', 'tf.argmin(input, axis)', "tf.argsort(values, axis, direction='DESCENDING', stable=True)", 'tf.eye(num_rows, dtype)', 'tf.math.cumsum(x, axis, exclusive=True)', 'tf.math.logical_not(x)', 'tf.math.negative(x)', 'tf.math.segment_min(data, segment_ids)', 'tf.math.top_k(input, k)', 'tf.math.unsorted_segment_min(data, segment_ids, num_segments)', 'tf.reverse(tensor, axis)', 'tf.roll(input, shift, axis)', 'tf.sign(x)', 'tf.unique_with_counts(x)', 'tf.exp(x)', 'tf.math.divide_no_nan(x, y)', 'tf.math.log(x)', 'tf.math.reciprocal(x)', 'tf.math.squared_difference(x, y)', "tf.pad(tensor, paddings, mode='CONSTANT', constant_values)", 'tf.reduce_prod(input_tensor, axis)', 'tf.repeat(input, repeats, axis)', 'tf.round(x)', 'tf.scatter_nd(indices, updates, shape)', 'tf.sort(values, axis)', 'tf.math.count_nonzero(input)', 'tf.math.count_nonzero(input, axis)', 'tf.math.segment_mean(data, segment_ids)', 'tf.math.unsorted_segment_mean(data, segment_ids, num_segments)', 'tf.range(start, limit, delta)', 'tf.repeat(input, repeats)', "tf.searchsorted(sorted_sequence, values, side='left')", "tf.searchsorted(sorted_sequence, values, side='right')", 'tf.sqrt(x)', 'tf.eye(num_rows, num_columns)', 'tf.math.logical_xor(x, y)', 'tf.math.reciprocal_no_nan(x)', 'tf.math.segment_prod(data, segment_ids)', 'tf.math.unsorted_segment_prod(data, segment_ids, num_segments)', "tf.pad(tensor, paddings, mode='REFLECT')", "tf.pad(tensor, paddings, mode='SYMMETRIC')", "tf.sort(values, axis, direction='DESCENDING')"]

SPARSETF_OPERATORS_WEIGHT_ORDER = ['tf.SparseTensor(indices, values, dense_shape)', 'tf.sparse.from_dense(tensor)', 'tf.sparse.to_dense(sp_input)', 'tf.sparse.expand_dims(sp_input, axis)', 'tf.sparse.reduce_max(sp_input, axis, output_is_sparse)', 'tf.sparse.reduce_sum(sp_input, axis, output_is_sparse)', 'tf.sparse.add(a, b)', 'tf.sparse.maximum(sp_a, sp_b)', 'tf.sparse.slice(sp_input, start, size)', 'tf.sparse.split(sp_input, num_split, axis)', 'tf.sparse.retain(sp_input, to_retain)', 'tf.sparse.to_dense(sp_input, default_value)', 'tf.sparse.transpose(sp_input)', 'tf.sparse.concat(axis, sp_inputs)', 'tf.sparse.minimum(sp_a, sp_b)', 'tf.sparse.reset_shape(sp_input)', 'tf.sparse.reshape(sp_input, shape)', 'tf.sparse.to_indicator(sp_input, vocab_size)', 'tf.sparse.transpose(sp_input, perm)']

In [9]:
def make_operators_section(shuffle = True, include_sparse = True, order_by_weight = False) -> str:
    operators = TFOPERATORS_WEIGHT_ORDER if order_by_weight else TFOPERATORS
    sparse_operators = SPARSETF_OPERATORS_WEIGHT_ORDER if order_by_weight else SPARSETF_OPERATORS
    shuffled_tf = operators.copy()
    shuffled_sparse = sparse_operators.copy()
    if shuffle:
        random.shuffle(shuffled_tf)
        random.shuffle(shuffled_sparse)
    
    tf_str = "\n".join(shuffled_tf)
    sparse_str = "\n".join(shuffled_sparse)

    ans = f"[TENSORFLOW OPERATORS]\n{tf_str}\n"
    if include_sparse:
        ans += f"\n[SPARSE TENSORFLOW OPERATORS]\n{sparse_str}"
    return ans

In [50]:
def make_in_context_example(task_json: TaskJSON, include_program = True) -> str:
    parsed_examples = Example.from_json(task_json["examples"])
    inputs_str = ""
    for inp in parsed_examples.inputs:
        inputs_str += f"{inp}\n"
    outputs_str = f"{parsed_examples.output}\n"

    formals_str = [f"in{i+1}" for i in range(len(parsed_examples.inputs))]

    ans = f"""[TASK DESCRIPTION]
{task['description']}

[INPUTS]
{inputs_str}
[OUTPUTS]
{outputs_str}
"""
    ans += f"""[PROGRAM]
def transform({", ".join(formals_str)}):
    """
    if include_program:
        ans += f"return {task_json['target_program']}\n"
    
    return ans

IN_CONTEXT_SECTION = "\n\n".join([make_in_context_example(task_json) for task_json in IN_CONTEXT_TASK_JSONS])

print(IN_CONTEXT_SECTION)


[TASK DESCRIPTION]
create a binary matrix where a specified column is set to one

[INPUTS]
[[0. 1. 0. 0.]
 [0. 1. 1. 0.]
 [1. 1. 1. 1.]]

[OUTPUTS]
[[0.   1.   0.   0.  ]
 [0.   0.5  0.5  0.  ]
 [0.25 0.25 0.25 0.25]]

[PROGRAM]
def transform(in1):
    return tf.divide(in1, tf.expand_dims(tf.reduce_sum(in1, axis=1), 1))


[TASK DESCRIPTION]
create a binary matrix where a specified column is set to one

[INPUTS]
[10 20  0 40  0 30]
[1 1 0 1 0 1]

[OUTPUTS]
[10 20 40 30]

[PROGRAM]
def transform(in1, in2):
    return tf.boolean_mask(in1, tf.cast(in2, tf.bool))


[TASK DESCRIPTION]
create a binary matrix where a specified column is set to one

[INPUTS]
[[[ 8  4  6]
  [ 2 12  3]]

 [[11 12  5]
  [ 9 12 12]]

 [[ 9  2 13]
  [ 7  0  7]]

 [[ 2 10  5]
  [ 7  1  2]]]

[OUTPUTS]
[[[ 8  4  6]
  [11 12  5]
  [ 9  2 13]
  [ 2 10  5]]

 [[ 2 12  3]
  [ 9 12 12]
  [ 7  0  7]
  [ 7  1  2]]]

[PROGRAM]
def transform(in1):
    return tf.cast(tf.unstack(in1, axis=1), tf.int32)



In [51]:
SYSTEM_PROMPT = """You are a coding assistant. Be precise and terse.
You will be provided a list of tensorflow operators, a task description, and some input/output examples.
Your task is to generate the body of a python function that will transform the input to the output.
Only use the operators provided in the list.
"""

def make_user_message(task, shuffle_operators = True, order_by_weight = False):
    parsed_examples: Example = task['parsed_examples']
    include_sparse = any(isinstance(i, tf.SparseTensor) for i in parsed_examples.inputs) or isinstance(parsed_examples.output, tf.SparseTensor)

    return f"""{make_operators_section(shuffle_operators, include_sparse, order_by_weight)}

{IN_CONTEXT_SECTION}

{make_in_context_example(task, include_program=False)}"""

In [52]:
def prompt(system_message: str, user_message: str, n_completions: int) -> t.List[str]:
    response = OPENAI.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ],
        n=n_completions,
        temperature=1.0,
    )
    return [choice.message.content for choice in response.choices]

In [54]:
TEST_TASK = TASK_JSONS["google_03"]

print(make_user_message(TEST_TASK, False, True))

prompt(SYSTEM_PROMPT, make_user_message(TEST_TASK), 5)

[TENSORFLOW OPERATORS]
tf.cast(x, dtype)
tf.expand_dims(input, axis)
tf.constant(value)
tf.squeeze(input, axis)
tf.constant(value, dtype)
tf.equal(x, y)
tf.gather(params, indices)
tf.greater(x, y)
tf.matmul(a, b)
tf.maximum(x, y)
tf.multiply(x, y)
tf.reduce_max(input_tensor)
tf.reduce_max(input_tensor, axis)
tf.reduce_sum(input_tensor)
tf.reduce_sum(input_tensor, axis)
tf.tensordot(a, b, axes)
tf.transpose(a)
tf.where(condition)
tf.where(condition, x, y)
tf.add(x, y)
tf.boolean_mask(tensor, mask)
tf.divide(x, y)
tf.gather_nd(params, indices)
tf.one_hot(indices, depth)
tf.range(start)
tf.reshape(tensor, shape)
tf.square(x)
tf.subtract(x, y)
tf.tile(input, multiples)
tf.argmax(input, axis)
tf.greater_equal(x, y)
tf.minimum(x, y)
tf.sequence_mask(lengths)
tf.zeros_like(input)
tf.concat(values, axis)
tf.gather_nd(params, indices, batch_dims)
tf.ones_like(input)
tf.shape(input)
tf.stack(values, axis)
tf.squeeze(input)
tf.abs(x)
tf.argsort(values, axis, stable=True)
tf.eye(num_rows)
tf.fill(

['return tf.sparse.slice(in1, [0,0,0], [1,-1,-1])',
 'return tf.sparse.slice(in1, start=[0, 0, 0], size=[1, 2, 800])',
 'return tf.sparse.slice(in1, [0,0,0], [1, -1, -1])',
 'return tf.sparse.slice(in1, [0, 0, 0], [1, -1, -1])',
 'return tf.sparse.slice(in1, [0,0,0], [1,2,800])']

In [55]:
for task_name, task in TASK_JSONS.items():
    if "completions" in task:
        del task["completions"]

In [67]:
NUM_COMPLETIONS = 10
i = 0
for task_name, task in TASK_JSONS.items():
    if "completions" in task:
        continue 
    print(f"Prompting for task: {task['name']}, description: {task['description']}")
    completions = prompt(SYSTEM_PROMPT, make_user_message(task, shuffle_operators=False, order_by_weight=True), NUM_COMPLETIONS)
    task["completions"] = completions
    pprint(completions)
    print()
    # i += 1
    # if i > 10:
    #     break



Prompting for task: google_14, description: circular buffer
['return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=1, axis=1)',
 'return tf.roll(in1, shift=-1, axis=-1)']

Prompting for task: google_15, description: pad a zero column
['return tf.pad(in1, [[0,0],[0,1]])',
 'return tf.pad(in1, [[0,0],[0,1]])',
 'return tf.pad(in1, [[0, 0], [0, 1]])',
 'return tf.pad(in1, [[0,0], [0,1]])',
 'return tf.pad(in1, [[0, 0], [0, 1]])',
 'return tf.pad(in1, [[0,0], [0,1]])',
 'return tf.pad(in1, [[0, 0], [0, 1]])',
 'return tf.pad(in1, [[0, 0], [0, 1]])',
 'return tf.pad(in1, [[0, 0], [0, 1]])',
 'return tf.pad(in1, [[0, 0], [0, 1]])']

Prompting for task: google_16, description: replicate elements a

In [68]:
TASKS_WITH_COMPLETIONS_FILE = CURRENT_DIRECTORY / "tfcoder_dataset_with_completions.json"
for name, task in TASK_JSONS.items():
    if "parsed_examples" in task:
        del task["parsed_examples"]
TASKS_WITH_COMPLETIONS_FILE.write_text(json.dumps(DATASET, indent=4))
for name, task in TASK_JSONS.items():
    task["parsed_examples"] = Example.from_json(task["examples"])

## extract tf operators

In [69]:
def extract_tf_operators(code_snippet):
    pattern = r"tf\.[a-zA-Z_][a-zA-Z0-9_]*(?:\.[a-zA-Z_][a-zA-Z0-9_]*)*"
    return set(re.findall(pattern, code_snippet))

In [70]:
def calculate_tf_operator_coverage_and_count(target_operators, completion_operators):
    """Extend to include all completion operators and mark those used in the target program."""
    completion_operators_count = Counter(completion_operators)
    tf_operators_dict = {op: completion_operators_count[op] for op in completion_operators_count}

    # Calculate coverage based on target program operators found in completions
    covered_operators = set(target_operators).intersection(completion_operators)
    coverage_percentage = len(covered_operators) / len(target_operators) * 100 if target_operators else 0

    return {
        "tf_operators": tf_operators_dict,
        "coverage_percentage": coverage_percentage,
        "total_in_target": len(target_operators),
        "total_covered": len(covered_operators)
    }

In [71]:
for name, task in TASK_JSONS.items():
    if "completions" not in task:
        continue
    completions = task["completions"]
    completion_tf_operators = [op for completion in completions for op in extract_tf_operators(completion)]
    target_program = task["target_program"]
    target_tf_operators = extract_tf_operators(target_program)
    tf_operator_info = calculate_tf_operator_coverage_and_count(target_tf_operators, completion_tf_operators)

    task["response"] = {
        "task_id": task.get("name", "unknown"),
        "completions": completions,
        "target-program": task["target_program"],
        "description": task["description"],
        **tf_operator_info  # Includes adjusted tf_operators dictionary
    }

In [72]:
TASKS_WITH_COMPLETIONS_FILE = CURRENT_DIRECTORY / "tfcoder_dataset_with_completions.json"
for name, task in TASK_JSONS.items():
    if "parsed_examples" in task:
        del task["parsed_examples"]
TASKS_WITH_COMPLETIONS_FILE.write_text(json.dumps(DATASET, indent=4))
for name, task in TASK_JSONS.items():
    task["parsed_examples"] = Example.from_json(task["examples"])

In [73]:
COVERAGE_PERCENTAGES = [task["response"]["coverage_percentage"] for task in list(TASK_JSONS.values())]

In [74]:
# average coverage percentage
sum(COVERAGE_PERCENTAGES) / len(COVERAGE_PERCENTAGES)

71.30434782608695

In [75]:
# median coverage percentage
sorted_coverage_percentages = sorted(COVERAGE_PERCENTAGES)
median_index = len(sorted_coverage_percentages) // 2
sorted_coverage_percentages[median_index]

100.0

In [76]:
OLD_OUTPUT_FILE = CURRENT_DIRECTORY / "output_tfcoder.old.json"
OLD_OUTPUT_JSON = json.loads(OLD_OUTPUT_FILE.read_text())

OLD_COVERAGE_PERCENTAGES = [task["coverage_percentage"] for task in OLD_OUTPUT_JSON]
sum(OLD_COVERAGE_PERCENTAGES) / len(OLD_COVERAGE_PERCENTAGES)

63.95833333333334

In [77]:
# median coverage percentage
sorted_old_coverage_percentages = sorted(OLD_COVERAGE_PERCENTAGES)
median_index = len(sorted_old_coverage_percentages) // 2
sorted_old_coverage_percentages[median_index]

75.0